<a href="https://colab.research.google.com/github/unitn-course-project/deep-learning/blob/main/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.insert(0, '/content/drive/MyDrive/DeepLearning/')
from loading_data import *
from model import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
transform = list()
transform.append(transforms.ToTensor())
transform.append(transforms.Normalize(mean=[0.5], std=[0.5]))
transform = transforms.Compose(transform)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PATH = "/content/drive/MyDrive/DeepLearning/checkpoint/model_VGG16_original.pt"
with torch.no_grad():
  full_test_data = PedestrianDataset("/content/drive/MyDrive/DeepLearning/datasets/annotations_train_v2.csv", "/content/drive/MyDrive/DeepLearning/datasets/val_attr", transform)
  test_loader = torch.utils.data.DataLoader(full_test_data, 32, shuffle=False)
  net = initialize_vgg16(32).to(device)
  # net = VGG16().to(device)
  if device == torch.device("cpu"):
      checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
  else:
    checkpoint = torch.load(PATH)
  net.load_state_dict(checkpoint['model_state_dict'])
  net.eval()
  cumulative_accuracy = torch.zeros(32).to(device)
  with tqdm(total=len(test_loader)) as pbar:
    for batch_idx, (inputs, targets) in enumerate(test_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)    
      outputs = net(inputs)
      predicted = outputs > 0.5
      acc = torch.sum(predicted.eq(targets),0)
      cumulative_accuracy += acc
      pbar.update(1)
    print(cumulative_accuracy/len(full_test_data))
    print(torch.mean(cumulative_accuracy/len(full_test_data)))

100%|██████████| 41/41 [02:11<00:00,  3.20s/it]

tensor([0.7308, 0.7562, 0.8862, 0.9177, 0.9108, 0.9362, 0.8292, 0.9723, 0.8523,
        0.8615, 0.8954, 0.8954, 0.9646, 0.9208, 0.8854, 0.9115, 0.9238, 0.8685,
        0.9315, 0.9631, 1.0000, 0.9915, 0.8454, 0.7985, 0.9723, 0.9185, 0.9908,
        0.8408, 0.8654, 0.9846, 0.9169, 0.9500])
tensor(0.9027)


scratch-> tensor([0.7308, 0.7562, 0.8862, 0.9346, 0.9215, 0.9362, 0.8823, 0.9723, 0.9162,
        0.8615, 0.7200, 0.8954, 0.9646, 0.9208, 0.8854, 0.9115, 0.9238, 0.9192,
        0.9315, 0.9631, 1.0000, 0.9915, 0.8454, 0.7985, 0.9723, 0.9185, 0.9908,
        0.8885, 0.9023, 0.9846, 0.9169, 0.9500])
tensor(0.9060)

origin -> tensor([0.7308, 0.7562, 0.8862, 0.9162, 0.9162, 0.9362, 0.7954, 0.9723, 0.8438,
        0.8615, 0.8885, 0.8954, 0.9646, 0.9208, 0.8854, 0.9115, 0.9238, 0.8531,
        0.9315, 0.9631, 1.0000, 0.9915, 0.8454, 0.7985, 0.9723, 0.9185, 0.9908,
        0.8408, 0.8654, 0.9846, 0.9169, 0.9500])
tensor(0.9008)

agumented -> tensor([0.7308, 0.7562, 0.8862, 0.8600, 0.9215, 0.9362, 0.8985, 0.9723, 0.9269,
        0.8615, 0.9323, 0.9838, 0.9646, 0.9208, 0.8854, 0.9115, 0.9238, 0.9162,
        0.9315, 0.9631, 1.0000, 0.9915, 0.8454, 0.7985, 0.9723, 0.9185, 0.9908,
        0.8662, 0.8808, 0.9846, 0.9169, 0.9500])
tensor(0.9125)

In [ ]:
from torch.autograd import Variable

transform = list()
transform.append(transforms.ToTensor())
transform.append(transforms.Normalize(mean=[0.5], std=[0.5]))
transform = transforms.Compose(transform)

full_training_data = datasets.ImageFolder('/content/drive/MyDrive/DeepLearning/datasets/val_iden/', transform=transform)
test_loader = torch.utils.data.DataLoader(full_test_data, 32, shuffle=False)
sample_image, sample_label = full_training_data[0]
image = Variable(sample_image, requires_grad=True)
activation = {}
def get_activation(name):
  def hook(model, input, output):
    activation[name] = output.detach()
  return hook

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PATH = "/content/drive/MyDrive/DeepLearning/checkpoint/model_iden_VGG16.pt"
with torch.no_grad():
  net = initialize_vgg16(751).to(device)
  net.classifier[3].register_forward_hook(get_activation('features'))
  if device == torch.device("cpu"):
      checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
  else:
    checkpoint = torch.load(PATH)
  net.load_state_dict(checkpoint['model_state_dict'])
  net.eval()
  for batch_idx, (inputs, targets) in enumerate(test_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)    
      outputs = net(inputs)
  # output = net(image)
      print("feature", len(activation['features']), len(activation['features'][0]))